# Практическая работа №3
## по предмету "Системы искусственного интеллекта"
### Выполнила студентка группы БВТ2101 Пьянова Анна

Целью практической работы является изучение методов регрессии.

В данной работе вам необходимо:
1. используя библиотеку sklearn, обучить линейную регрессию без использования регуляризации
2. изучить работу класса Lasso для регуляризации, подобрать наилучший параметр для данного набора данных.
3. изучить работу класса Ridge для регуляризации, подобрать наилучший параметр альфа для данного набора данных.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import warnings
from sklearn.exceptions import ConvergenceWarning

Предобработка данных

In [2]:
data = pd.read_csv('processor.csv')

X = data.drop('Price', axis=1)
y = data['Price']

# Определение числовых и категориальных признаков
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Препроцессор для числовых признаков
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# Препроцессор для категориальных признаков
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Объединение препроцессоров
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Применение препроцессора
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)


Линейная регрессия без регуляризации

In [3]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

y_val_pred_linear = linear_reg.predict(X_val)
mse_linear_val = mean_squared_error(y_val, y_val_pred_linear)
y_test_pred_linear = linear_reg.predict(X_test)
mse_linear_test = mean_squared_error(y_test, y_test_pred_linear)
print(f"MSE for Linear Regression (val): {mse_linear_val}")
print(f"MSE for Linear Regression (test): {mse_linear_test}")

MSE for Linear Regression (val): 428396492.2859303
MSE for Linear Regression (test): 564981862.231206


Линейная регрессия с L1-регуляризцией

In [7]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)
lasso = Lasso()
param_grid_lasso = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search_lasso = GridSearchCV(lasso, param_grid_lasso, cv=5, scoring='neg_mean_squared_error')
grid_search_lasso.fit(X_train, y_train)

best_alpha_lasso = grid_search_lasso.best_params_['alpha']
print(f"Best alpha for Lasso: {best_alpha_lasso}")

y_val_pred_lasso = grid_search_lasso.predict(X_val)
mse_lasso_val = mean_squared_error(y_val, y_val_pred_lasso)
y_test_pred_lasso = grid_search_lasso.predict(X_test)
mse_lasso_test = mean_squared_error(y_test, y_test_pred_lasso)
print(f"MSE for Lasso Regression (val): {mse_lasso_val}")
print(f"MSE for Lasso Regression (test): {mse_lasso_test}")

Best alpha for Lasso: 100
MSE for Lasso Regression (val): 429608292.63583606
MSE for Lasso Regression (test): 562570791.1889168


Линейная регрессия с L2-регуляризацией

In [5]:
ridge = Ridge()
param_grid_ridge = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search_ridge = GridSearchCV(ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error')
grid_search_ridge.fit(X_train, y_train)

best_alpha_ridge = grid_search_ridge.best_params_['alpha']
print(f"Best alpha for Ridge: {best_alpha_ridge}")

y_val_pred_ridge = grid_search_ridge.predict(X_val)
mse_ridge_val = mean_squared_error(y_val, y_val_pred_ridge)
y_test_pred_ridge = grid_search_ridge.predict(X_test)
mse_ridge_test = mean_squared_error(y_test, y_test_pred_ridge)
print(f"MSE for Ridge Regression (val): {mse_ridge_val}")
print(f"MSE for Ridge Regression (test): {mse_ridge_test}")

Best alpha for Ridge: 1
MSE for Ridge Regression (val): 405063327.4651937
MSE for Ridge Regression (test): 551813208.143868
